In [1]:
# install missing packages
%pip install plotly
%pip install py-cpuinfo

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# check system details
import os
import psutil
import cpuinfo

try:
    ram_info = psutil.virtual_memory()
    print(f"Total RAM: {ram_info.total / 1024 / 1024 / 1024:.2f} GB")
    print(f"Available RAM: {ram_info.available / 1024 / 1024 / 1024:.2f} GB")
    print(f"Used RAM: {ram_info.used / 1024 / 1024 / 1024:.2f} GB")
    print(f"Percentage Usage Of RAM: {ram_info.percent}%")
    print(f"CPU Cores: {os.cpu_count()}")
    print(f"CPU Speed: {cpuinfo.get_cpu_info()['hz_actual_friendly']}")
except:
    print("RAM and CPU info not available on this system")

Total RAM: 15.47 GB
Available RAM: 13.33 GB
Used RAM: 1.86 GB
Percentage Usage Of RAM: 13.9%
CPU Cores: 4
CPU Speed: 2.5000 GHz


In [3]:
# import requirements
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from regression import Regression

In [4]:
# get the data
riders = pd.read_csv("riders.csv")
riders = riders.sort_values(by=["date", "route", "daytype"]).reset_index(drop=True)
riders = riders.head(int(0.25 * riders.shape[0]))  # only use 25% of the data

In [5]:
# split up the data into training and testing
y = riders[["rides"]]
X = riders.drop(columns="rides")
trainX = X.head(int(0.8 * X.shape[0]))
trainy = y.head(int(0.8 * y.shape[0]))
testX = X.tail(int(0.2 * X.shape[0])).reset_index(drop=True)
testy = y.tail(int(0.2 * y.shape[0])).reset_index(drop=True)

In [6]:
# build the model
print("\n---- Riders Regression Analysis ----\n")
model = Regression(name="Riders Regression Analysis", frac=0.33)  # only use 33% of the data for preprocessing
try:
    model.load()  # load the machine learning pipeline
    predictions = model.predict(testX)
except:
    model.fit(trainX, trainy)  # build the machine learning pipeline
    predictions = model.predict(testX)


---- Riders Regression Analysis ----

1/6) Model Training
> Renaming Features
> Extracting Time Features
> Transforming Categorical Features
> Renaming Features
> Filling In Missing Values
> Removing Constant Features
> Scaling Features
> Selecting Features
> Computing Atwoood Numbers
> Binning Features
> Computing Reciprocals
> Computing Interactions
> Removing Constant Features
> Selecting Features
> Transforming The Training Data
> Training XGBoost
2.84 Minutes
2/6) Model Performance
> Transforming The Testing Data
20.54 Seconds
3/6) Model Deployment
> Transforming All The Data
> Training XGBoost
2.06 Minutes
4/6) Model Indicators
0.14 Seconds
5/6) Model Prediction
> Transforming The New Data
5.48 Seconds
6/6) Model Monitoring
17.97 Seconds


In [7]:
# model diagnostics
print("Model Diagnostics:")
print(f"> RMSE: {model.rmse}")
print(f"> R2: {model.r2}")
print(f"> In Control: {model.in_control}")
print("> Model Indicators:")
for i, indicator in enumerate(model.indicators["Indicator"][:10]):
    print(f"{i+1}. {indicator}")
print("> Feature Drift:")
for i, feature in enumerate(model.drift.loc[model.drift["pvalue"] < 0.05, "Feature"][:10]):
    print(f"{i+1}. {feature}")
if model.drift.loc[model.drift["pvalue"] < 0.05].shape[0] == 0:
    print("None")

Model Diagnostics:
> RMSE: 3702.392466284922
> R2: 0.6567789666858564
> In Control: 98.37%
> Model Indicators:
1. route_79
2. route_82*daytype_W
3. route_66*daytype_W
4. route_9
5. route_8*daytype_W
6. route_20*daytype_W
7. route_49
8. route_22
9. route_4
10. route_53*daytype_W
> Feature Drift:
1. daytype_W
2. daytype_U
3. rides


In [8]:
# score the model
rmse = mean_squared_error(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
    squared=False,
)
r2 = r2_score(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
)

print(f"RMSE: {rmse}")
print(f"R2: {r2}")

RMSE: 3731.1336220059716
R2: 0.6414234203579768


In [9]:
# save the machine learning pipeline
model.dump()

In [10]:
# refit the model to include the test data
model.refit(testX, testy)

Model Retraining:
> Transforming The New Data
> Training XGBoost
1.98 Minutes
